Sec 1: Transformation from geographic projection coordinates to longtitude/latitude bounding box

In [1]:
from pystac_client import Client
import pandas as pd
import geopandas as gpd
from shapely.geometry import shape, mapping
import logging

# Set up logging to track API calls
logging.basicConfig()
logger = logging.getLogger('pystac_client')
logger.setLevel(logging.INFO)

# Read the shapefile to determine the bounding box
import geopandas as gpd
from pyproj import CRS

proj_wkt2 = "PROJCS[\"Mars (2015) - Sphere / Ocentric / Equirectangular, clon = 0\",GEOGCS[\"Mars (2015) - Sphere / Ocentric\",DATUM[\"Mars (2015) - Sphere\",SPHEROID[\"Mars (2015) - Sphere\",3396190,0]],PRIMEM[\"Reference Meridian\",0],UNIT[\"degree\",0.0174532925199433,AUTHORITY[\"EPSG\",\"9122\"]]],PROJECTION[\"Equirectangular\"],PARAMETER[\"standard_parallel_1\",0],PARAMETER[\"central_meridian\",0],PARAMETER[\"false_easting\",0],PARAMETER[\"false_northing\",0],UNIT[\"metre\",1,AUTHORITY[\"EPSG\",\"9001\"]],AXIS[\"Easting\",EAST],AXIS[\"Northing\",NORTH]]"

# Define the target Mars lon/lat coordinate system
mars_lonlat = CRS.from_proj4(
    "+proj=longlat +a=3396190 +b=3396190 +no_defs"
)
# Load the shapefile
shp_file = "roi_test.shp"
gdf = gpd.read_file(shp_file)
# Reproject the GeoDataFrame
gdf.to_crs(mars_lonlat, inplace=True)
bounding_boxes = gdf.geometry.bounds.values
print("Transformed Bounding Box (Target Projection):", bounding_boxes)

Transformed Bounding Box (Target Projection): [[76.97670509 17.72565618 77.16420794 17.9416318 ]]


Sec 2: Perfrom query of link resource using API from USGS

In [2]:
output_file = "output_data_test.gpkg"

# Set up the STAC catalog client
catalog_url = "https://stac.astrogeology.usgs.gov/api/"
catalog = Client.open(catalog_url)

# database to query
database = 'mro_ctx_controlled_usgs_dtms'

# Simplify and convert geometry to GeoJSON once
tolerance = 90  # Simplify geometry with a tolerance of 90 meters
simplified_geometries = [mapping(geometry.simplify(tolerance, preserve_topology=True)) for geometry in gdf.geometry]

# Initialize a set to track seen IDs and a list to store unique items
seen_ids = set()
items = [
    {**feature, 'geometry': shape(feature['geometry'])}
    for geojson_geometry in simplified_geometries
    for feature in catalog.search(collections=[database], intersects=geojson_geometry)
                         .item_collection_as_dict()['features']
    if feature['id'] not in seen_ids and not seen_ids.add(feature['id'])
]
'''
# Perform the database query in batches, if possible, to minimize repeated calls
items = []
for geojson_geometry in simplified_geometries:
    # Perform the search once for each geometry
    result = catalog.search(collections=[database], intersects=geojson_geometry, max_items=3)
    feature_list = result.item_collection_as_dict()['features']  # List of features from the search result
    # Collect and process features from search results
    for feature in feature_list:
        # Convert geometry to Shapely shape and append to items list
        feature['geometry'] = shape(feature['geometry'])
        items.append(feature)
'''

# Convert to a GeoDataFrame and set the custom CRS
items = gpd.GeoDataFrame(pd.json_normalize(items), crs=mars_lonlat)  # Convert to GeoDataFrame

# Save the GeoDataFrame to a .gpkg file
items.to_file(output_file)

print(f"File saved to {output_file}")
# Verify the retrieved items
print(f"Number of items retrieved: {items.shape[0]}")

File saved to output_data_test.gpkg
Number of items retrieved: 32


In [3]:
# Write URLs to a text file
output_file = "ctx_dtm_to_download.txt"
with open(output_file, 'w') as f:
    for _, row in items.iterrows():  # Iterate over rows of the GeoDataFrame
        # Extract the URLs from the row
        url1 = row['assets.image.href']
        url2 = row['assets.orthoimage.href']
        url3 = row['assets.dtm.href']
        
        # Ensure the URL strings are correctly formatted before writing (if needed)
        updated_url = f"{url1}\n{url2}\n{url3}\n"
        
        # Write the URLs to the file
        f.write(updated_url)

print(f"URLs have been written to '{output_file}'.")

URLs have been written to 'ctx_dtm_to_download.txt'.
